In [1]:
item = {'Image_Path': '/scratch/faaraan/LLaVAData/images/week_03/week_03_page_117.png'}

# Extract the filename
image_id = item['Image_Path'].split('/')[-1]

print(image_id)

week_03_page_117.png


In [3]:
import json
import pandas as pd
import os
# Load the JSON data
with open('Final_weeks_QnA.json', 'r') as f:
    data = json.load(f)

image_folder = '/scratch/faaraan/LLaVAData/images/'

Q_A = pd.read_json("Final_weeks_QnA.json")   
Q_A['Image_Path'] = [os.path.join(image_folder, f"week_{week:02d}/week_{week:02d}_page_{page:03d}.png") for week, page in zip(Q_A['week'], Q_A['page'])]
# Define the output data structure
result = []

# Iterate through each item in the input JSON
for i, item in Q_A.iterrows():
    # Generate the ID and image path
    # id_ = f"week_{item['week']:02d}_page_{item['page']:03d}"
    # image = f"week_{item['week']:02d}/week_{item['week']:02d}_page_{item['page']:03d}.png"
    
    # if item['page'] < 10:
    #     image = f'week_03/week_03_page_00{item["page"]}.png'
    #     id_ = f'week_03_page_00{item["page"]}.png'
    # elif item['page'] > 100:
    #     image = f'week_03/week_03_page_{item["page"]}.png'
    #     id_ = f'week_03_page_{item["page"]}.png'

    # else:
    #     image = f'week_03/week_03_page_0{item["page"]}.png'
    #     id_ = f'week_03_page_0{item["page"]}.png'

    image = item['Image_Path']
    id_ = item['Image_Path'].split('/')[-1]

    # Construct the conversation
    conversation = [
        {
            "from": "human",
            "value": f"<image>\n{item['instruction']}"
        },
        {
            "from": "gpt",
            "value": f"{item['response']}"
        }
    ]
    
    # Append the conversation to the result
    result.append({"id": id_, "image": image, "conversations": conversation})

# Write the result to a new JSON file
with open('result_data_llava.json', 'w') as f:
    json.dump(result, f, indent=2)

In [2]:
!pip install rouge

In [10]:
import json
import pandas as pd
from rouge import Rouge

# Load JSON data
input_file = 'predictions_week_1_to_8.json'  # Adjust the path to your JSON file
with open(input_file, 'r') as f:
    data = json.load(f)

# Clean the predicted responses
for item in data:
    item["Predicted"] = item["Predicted"].replace("\n", "").replace("\r", "").strip()
    # Clean other unwanted characters or text here as needed
    item["Predicted"] = item["Predicted"].replace("\u0131", "")
    item["Predicted"] = item["Predicted"].replace("\u0627", "")
    item["Predicted"] = item["Predicted"].replace("\u0646", "")
    item["Predicted"] = item["Predicted"].replace(
        "\u0627\u0646\u0635\u062d \u0628\u062a\u062e\u0641\u064a\u0641 \u0627\u0644\u0635\u0648\u0631\u0629 \u0645\u0646\u0639\u062f \u0627\u0644\u0642\u064a\u0627\u0645 \u0628\u0639\u0645\u0644 \u0627\u0644\u0646\u0635\u062d \u0628\u062a\u062e\u0641\u064a\u0641 \u0627\u0644\u0635\u0648\u0631\u0629 \u0628\u0639\u062f \u0627\u0644\u0642\u064a\u0627\u0645 \u0628\u0639\u0645\u0644\u064a\u0629 \u0639\u0646\u0642\u064a\u0636\n\n", "")

# Function to calculate ROUGE scores
def calculate_rouge(predicted_text, ground_truth_text):
    rouge = Rouge()
    try:
        scores = rouge.get_scores(predicted_text, ground_truth_text)
        rouge_1_score = scores[0]["rouge-1"]["f"]
        rouge_2_score = scores[0]["rouge-2"]["f"]
        rouge_l_score = scores[0]["rouge-l"]["f"]
    except ValueError as e:
        print(f"ValueError: {e} - Assigning zero scores.")
        rouge_1_score, rouge_2_score, rouge_l_score = 0.0, 0.0, 0.0
    return rouge_1_score, rouge_2_score, rouge_l_score

# Initialize list to store ROUGE scores
rouge_scores = []

# Iterate over the data and calculate ROUGE scores
for entry in data:
    question = entry['Question']
    original_response = entry['Ground Truth'].strip()
    predicted_response = entry['Predicted'].strip()
    
    # Skip if the predicted or ground truth response is empty
    if not predicted_response or not original_response:
        print(f"Skipping empty prediction or ground truth for question: {question}")
        continue

    # Calculate ROUGE scores with exception handling
    rouge_1, rouge_2, rouge_l = calculate_rouge(predicted_response, original_response)
    rouge_scores.append({
        'Question': question,
        'Ground Truth': original_response,
        'Predicted': predicted_response,
        'ROUGE-1': rouge_1,
        'ROUGE-2': rouge_2,
        'ROUGE-L': rouge_l
    })

# Convert the list to a DataFrame
rouge_df = pd.DataFrame(rouge_scores)

# Calculate average ROUGE scores
average_rouge_scores = rouge_df[['ROUGE-1', 'ROUGE-2', 'ROUGE-L']].mean()

# Print average ROUGE scores
print("Average ROUGE-1 score:", average_rouge_scores['ROUGE-1'])
print("Average ROUGE-2 score:", average_rouge_scores['ROUGE-2'])
print("Average ROUGE-L score:", average_rouge_scores['ROUGE-L'])

# Optionally, save the DataFrame with ROUGE scores to a new JSON file
# output_file = '/path/to/results_with_rouge_scores.json'  # Adjust the path to your output JSON file
# rouge_df.to_json(output_file, orient='records', indent=4)
# print(f"Results with ROUGE scores saved to {output_file}")


ValueError: Hypothesis is empty. - Assigning zero scores.
ValueError: Hypothesis is empty. - Assigning zero scores.
ValueError: Hypothesis is empty. - Assigning zero scores.
ValueError: Hypothesis is empty. - Assigning zero scores.
Skipping empty prediction or ground truth for question: What operations are discussed in the transcript?
Skipping empty prediction or ground truth for question: What operations are outlined in the transcript?
ValueError: Hypothesis is empty. - Assigning zero scores.
ValueError: Hypothesis is empty. - Assigning zero scores.
ValueError: Hypothesis is empty. - Assigning zero scores.
ValueError: Hypothesis is empty. - Assigning zero scores.
Average ROUGE-1 score: 0.31247366398542326
Average ROUGE-2 score: 0.13591904197166196
Average ROUGE-L score: 0.28565209414172926


In [1]:
import json
import pandas as pd
from rouge import Rouge

# Load JSON data
input_file = 'Sporrow_test_result_32_stage_4_epochs.json'  # Adjust the path to your JSON file
with open(input_file, 'r') as f:
    data = json.load(f)

# Remove newline characters from the predicted_response fields
for item in data:
    item["predicted_response"] = item["Predicted_Answer"].replace("\n", "")

# Function to calculate ROUGE scores
def calculate_rouge(predicted_text, ground_truth_text):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_text, ground_truth_text)
    rouge_1_score = scores[0]["rouge-1"]["f"]
    rouge_2_score = scores[0]["rouge-2"]["f"]
    rouge_l_score = scores[0]["rouge-l"]["f"]
    return rouge_1_score, rouge_2_score, rouge_l_score

# Initialize list to store ROUGE scores
rouge_scores = []

# Iterate over the data and calculate ROUGE scores
for entry in data:
    question = entry['instruction']
    original_response = entry['Original_Answer']
    predicted_response = entry['predicted_response']
    
    rouge_1, rouge_2, rouge_l = calculate_rouge(predicted_response, original_response)
    rouge_scores.append({
        'question': question,
        'original_response': original_response,
        'predicted_response': predicted_response,
        'ROUGE-1': rouge_1,
        'ROUGE-2': rouge_2,
        'ROUGE-L': rouge_l
    })

# Convert the list to a DataFrame
rouge_df = pd.DataFrame(rouge_scores)

# Calculate average ROUGE scores
average_rouge_scores = rouge_df[['ROUGE-1', 'ROUGE-2', 'ROUGE-L']].mean()

# Print average ROUGE scores
print("Average ROUGE-1 score:", average_rouge_scores['ROUGE-1'])
print("Average ROUGE-2 score:", average_rouge_scores['ROUGE-2'])
print("Average ROUGE-L score:", average_rouge_scores['ROUGE-L'])

# Optionally, save the DataFrame with ROUGE scores to a new JSON file
# output_file = '/path/to/results_with_rouge_scores.json'  # Adjust the path to your output JSON file
# rouge_df.to_json(output_file, orient='records', indent=4)
# print(f"Results with ROUGE scores saved to {output_file}")


Average ROUGE-1 score: 0.37998290227389137
Average ROUGE-2 score: 0.1698322163495669
Average ROUGE-L score: 0.33219930514546675
